In [20]:
import rdflib
import pandas as pd
from IPython.display import display
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON



In [3]:
# Load RDF graph
rdf_graph = rdflib.Graph()
rdf_graph.parse("LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl", format="ttl")

file:///c:/Users/Home/ARCS%20BIM%20Automation%20Project/BIM%20Casestudy/LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl does not look like a valid URI, trying to serialize this will break.
file:///c:/Users/Home/ARCS%20BIM%20Automation%20Project/BIM%20Casestudy/LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl does not look like a valid URI, trying to serialize this will break.


<Graph identifier=N8be97a61361c43af80e7dc58f27c4bae (<class 'rdflib.graph.Graph'>)>

In [5]:
# Getting Storey information 
with open("..\SPARQL\storeyElevation.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Pandas DataFrame
data = []
for row in results:
    data.append([str(row.label), str(row.elevationValue), str(row.Use)])

df = pd.DataFrame(data, columns=["Name", "Elevation", "Use"])
df

,Name,Elevation,Use
0,02-SECOND FLOOR,13.333333333333371,None
1,04-FOURTH FLOOR,35.33333333333337,None
2,03-THIRD FLOOR,24.33333333333337,None
3,01-GROUND FLOOR,0,Grade Plane


In [11]:
# Getting Areas information 
with open("..\SPARQL\\areasSpaces.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Pandas DataFrame
data = []
for row in results:
    # Convert Literal values to float
    area_value = float(row.areaValue)
    storey_area_value = float(row.storeyAreaValue)
    
        # Calculate percent area, round to integer, and format with '%'
    if storey_area_value != 0:
        percentArea = f"{int(round((area_value / storey_area_value) * 100))}%"
    else:
        percentArea = "N/A"  # Handle division by zero if needed
    
    data.append([str(row.levelValue), str(row.name), str(row.occupancyClassification), str(row.id), str(row.areaValue), str(row.storeyHeightValue), str(percentArea)])

df = pd.DataFrame(data, columns=["Level", "Name", "Occupancy", "ID", "Area", "Height", "Percent Area of Story"])

#hardcode accessory for now
df["Accessory"] = False
#df["Percent Area of Story"] = None
df

,Level,Name,Occupancy,ID,Area,Height,Percent Area of Story,Accessory
0,01,Ground Floor Gym,A-3,32254,825.0373394415645,0,4%,False
1,01,Ground Floor Banquet,A-2,32254,6108.644158203678,0,26%,False
2,01,Ground Floor Breakfast,A-2,32254,779.5649402682602,0,3%,False
3,01,Ground Floor Pool,A-3,32254,2086.562270762054,0,9%,False
4,01,Ground Floor Guest,R-1,32254,6006.713107638702,0,26%,False
5,01,Ground Floor Business,B,32254,3332.097216021819,0,14%,False
6,04,Fourth Floor,R-1,32254,15467.274522569236,35.33333333333337,103%,False
7,03,Third Floor,R-1,32254,15467.274522569236,24.33333333333337,103%,False
8,02,Second Floor,R-1,32254,15467.274522569367,13.333333333333371,101%,False
9,01,Ground Floor Public,B,32254,4366.970410816291,0,19%,False


In [4]:
# Getting Room information 
with open("..\SPARQL\\rooms.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Pandas DataFrame
data = []
for row in results:
    data.append([str(row.levelValue), str(row.numberAndName), str(row.roomType), str(row.areaValue), str(row.smrvCalcOccupantLoadValue), str(row.flame), str(row.function)])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Level", "Number Name", "Type", "Area", "Occupant Load", "Flame Spread Classification", "Function of Space"])
df["ID"] = None
df["Associated Area"] = None
df["Occupancy Classification"] = None

display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))


,Level,Number Name,Type,Area,Occupant Load,Flame Spread Classification,Function of Space,ID,Associated Area,Occupancy Classification
0,01,495/King Suite,Room,344.04253472221376,2,C,Residential,None,None,None
1,01,495/King Suite,Room,344.04253472221376,2,C,Residential,None,None,None
2,01,494/King Suite,Room,344.04253472220483,2,C,Residential,None,None,None
3,01,494/King Suite,Room,344.04253472220483,2,C,Residential,None,None,None
4,01,497/1 Bedroom Suite - ADA,Room,267.99999999997726,2,C,Residential,None,None,None
5,01,497/1 Bedroom Suite - ADA,Room,267.99999999997726,2,C,Residential,None,None,None
6,01,496/1 Bedroom Suite - ADA,Room,268.00000000001774,2,C,Residential,None,None,None
7,01,496/1 Bedroom Suite - ADA,Room,268.00000000001774,2,C,Residential,None,None,None
8,01,489/King Suite,Room,343.50694444447686,2,C,Residential,None,None,None
9,01,489/King Suite,Room,343.50694444447686,2,C,Residential,None,None,None


In [41]:
# Getting Exterior Walls information 
with open("../SPARQL/exteriorWalls.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Assuming your query results include width and height for each subelement (doors and windows) under each wall

# Prepare the results for DataFrame, including subelement dimensions for area calculations
data = []
for row in results:
    wall_id = str(row.wall) if row and row.wall else ""
    base_constraint = str(row.baseConstraintValue) if row and row.baseConstraintValue else ""
    top_constraint = str(row.topConstraintValue) if row and row.topConstraintValue else ""
    batid = str(row.batid) if row and row.batid else ""
    fire_rating = str(row.fireRating) if row and row.fireRating else ""
    facing = str(row.facingDirection) if row and row.facingDirection else ""
    smrv_exterior_fire_separation_distance = str(row.smrvExteriorFireSeparationDistanceValue) if row and row.smrvExteriorFireSeparationDistanceValue else ""
    unconnected_height = str(row.unconnectedHeightValue) if row and row.unconnectedHeightValue else ""
    length = str(row.lengthValue) if row and row.lengthValue else ""
    area = float(row.areaValue) if row and row.areaValue else 0  # Wall's total area
    base_elevation = str(row.baseElevationValue) if row and row.baseElevationValue else ""
    top_elevation = str(row.topElevationValue) if row and row.topElevationValue else ""
    
    # Retrieve subelement dimensions for area calculations
    width_window = float(row.widthWindowValue) if row.widthWindowValue else 0
    height_window = float(row.heightWindowValue) if row.heightWindowValue else 0
    width_door = float(row.widthDoorValue) if row.widthDoorValue else 0
    height_door = float(row.heightDoorValue) if row.heightDoorValue else 0
    
    # Calculate area of subelements and total for the wall
    area_window = width_window * height_window
    area_door = width_door * height_door
    total_opening_area = area_window + area_door
    
    # Calculate percentage area of openings if wall area is defined
    percentage_opening_area = (total_opening_area / area * 100) if area > 0 else None
    
    # Append all attributes for DataFrame creation
    data.append([
        base_constraint, top_constraint, batid, fire_rating, facing, smrv_exterior_fire_separation_distance,
        unconnected_height, length, area, base_elevation, top_elevation, total_opening_area, percentage_opening_area
    ])

# Create the DataFrame with the new calculated columns
df = pd.DataFrame(data, columns=[
    "Starts on Story", "Ends on Story", "ID", "Fire Rating", "Wall Direction Facing",
    "Distance to Property Line", "Unconnected Height", "Length", "Area", "Base Height", "Top Height",
    "Area of Openings", "Percentage Area of Openings"
])

# Process Base Constraint and Top Constraint to create Story Span
def extract_level_number(level_str):
    parts = level_str.split('-')
    if parts:
        try:
            return int(parts[0])
        except ValueError:
            return None
    return None

def create_story_span(row):
    base_constraint = row["Starts on Story"]
    top_constraint = row["Ends on Story"]
    
    base_level = extract_level_number(base_constraint)
    top_level = extract_level_number(top_constraint.replace("Up to level: ", ""))
    
    if base_level is not None and top_level is not None:
        return f"From {base_level} to {top_level}"
    return ""

df["Story Span"] = df.apply(create_story_span, axis=1)

# Additional hardcoded columns
df["Mark"] = "Not Set"
df["Stacked Wall"] = False
df["Wall Function Name"] = "exterior"
df["Exterior"] = True
df["Calculated Function"] = "exterior"
df["Use for Perimeter Calc"] = None

# Display the DataFrame
display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))


,Starts on Story,Ends on Story,ID,Fire Rating,Wall Direction Facing,Distance to Property Line,Unconnected Height,Length,Area,Base Height,Top Height,Area of Openings,Percentage Area of Openings,Story Span,Mark,Stacked Wall,Wall Function Name,Exterior,Calculated Function,Use for Perimeter Calc
0,01,02,369093,1,Wall Faces Unoccupied Space,200.9833182574975,13.333333333333371,12.075402025017581,124.485514,0,13.333333333333371,24.444444,19.636377,From 1 to 2,Not Set,False,exterior,True,exterior,None
1,01,02,369145,1,Wall Faces Unoccupied Space,202.6055474643932,13.333333333333371,19.060014641648966,216.795736,0,13.333333333333371,24.444444,11.275335,From 1 to 2,Not Set,False,exterior,True,exterior,None
2,01,02,369228,1,Wall Faces Unoccupied Space,125.89519812700523,13.333333333333371,2.6744567047879455,26.818299,0,13.333333333333371,0.000000,0.000000,From 1 to 2,Not Set,False,exterior,True,exterior,None
3,01,02,358135,1,Wall Faces Unoccupied Space,91.9382703059302,13.333333333333371,6.677083333334251,82.350694,0,13.333333333333371,0.000000,0.000000,From 1 to 2,Not Set,False,exterior,True,exterior,None
4,01,02,358096,1,Wall Faces Unoccupied Space,91.4364908821414,13.333333333333371,25.333333333333766,272.444444,0,13.333333333333371,20.000000,7.340946,From 1 to 2,Not Set,False,exterior,True,exterior,None
5,01,02,358096,1,Wall Faces Unoccupied Space,91.4364908821414,13.333333333333371,25.333333333333766,272.444444,0,13.333333333333371,20.000000,7.340946,From 1 to 2,Not Set,False,exterior,True,exterior,None
6,01,02,358014,1,Wall Faces Unoccupied Space,352.85824283462665,13.333333333333371,6.677083333332062,82.350694,0,13.333333333333371,0.000000,0.000000,From 1 to 2,Not Set,False,exterior,True,exterior,None
7,01,02,358242,1,Wall Faces Unoccupied Space,93.91596528318124,13.333333333333371,48.045576914928674,526.395449,0,13.333333333333371,20.000000,3.799425,From 1 to 2,Not Set,False,exterior,True,exterior,None
8,01,02,358242,1,Wall Faces Unoccupied Space,93.91596528318124,13.333333333333371,48.045576914928674,526.395449,0,13.333333333333371,20.000000,3.799425,From 1 to 2,Not Set,False,exterior,True,exterior,None
9,01,02,358242,1,Wall Faces Unoccupied Space,93.91596528318124,13.333333333333371,48.045576914928674,526.395449,0,13.333333333333371,20.000000,3.799425,From 1 to 2,Not Set,False,exterior,True,exterior,None


In [10]:
# Getting Interior Walls information 
with open("../SPARQL/interiorWalls.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Prepare the results for DataFrame
data = []
for row in results:
    wall_id = str(row.wall) if row and row.wall else ""
    base_constraint = str(row.baseConstraintValue) if row and row.baseConstraintValue else ""
    top_constraint = str(row.topConstraintValue) if row and row.topConstraintValue else ""
    batid = str(row.batid) if row and row.batid else ""
    fire_rating = str(row.fireRating) if row and row.fireRating else ""
    unconnected_height = str(row.unconnectedHeightValue) if row and row.unconnectedHeightValue else ""
    length = str(row.lengthValue) if row and row.lengthValue else ""
    area = str(row.areaValue) if row and row.areaValue else ""
    base_elevation = str(row.baseElevationValue) if row and row.baseElevationValue else ""
    top_elevation = str(row.topElevationValue) if row and row.topElevationValue else ""
    
    data.append([base_constraint, top_constraint, batid, fire_rating, unconnected_height, length, area, base_elevation, top_elevation])

df = pd.DataFrame(data, columns=[
    "Starts on Story", 
    "Ends on Story", 
    "ID", 
    "Fire Rating",
    "Unconnected Height", 
    "Length", 
    "Area",
    "Base Elevation", 
    "Top Elevation"
])

# Process Base Constraint and Top Constraint to create Story Span
def extract_level_number(level_str):
    # Extract the number from the level string
    parts = level_str.split('-')
    if parts:
        try:
            return int(parts[0])
        except ValueError:
            return None
    return None

def create_story_span(row):
    base_constraint = row["Starts on Story"]
    top_constraint = row["Ends on Story"]
    
    base_level = extract_level_number(base_constraint)
    top_level = extract_level_number(top_constraint.replace("Up to level: ", ""))
    
    if base_level is not None and top_level is not None:
        return f"From {base_level} to {top_level}"
    return ""

# Ensure `create_story_span` returns a string for each row
df["Story Span"] = df.apply(lambda row: create_story_span(row), axis=1)

# Adding additional columns
df["Mark"] = "Not Set"  # hardcoded value 
df["Stacked Wall"] = False  # hardcoded value
df["Wall Function Name"] = "interior" #hardcoded value
df["Exterior"] = False #hardcoded value 
df["Calculated Function"] = "interior" #hardcoded value
df["Bearing"] = False #hardcoded Value

display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))


,Starts on Story,Ends on Story,ID,Fire Rating,Unconnected Height,Length,Area,Base Elevation,Top Elevation,Story Span,Mark,Stacked Wall,Wall Function Name,Exterior,Calculated Function,Bearing
0,01,02,366868,1,13.333333333333371,29.59375000000361,386.80555555560494,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
1,01,02,366909,0,13.333333333333371,29.593749999998675,386.80555555553906,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
2,01,02,367060,0,13.333333333333371,22.916666666664497,297.77777777774963,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
3,01,02,367101,0,13.333333333333371,22.916666666664486,255.7777777777497,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
4,01,02,367022,0,13.333333333333371,29.593749999998682,386.80555555553906,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
5,01,02,366678,0,13.333333333333371,22.916666666667954,297.7777777777958,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
6,01,02,366630,0,13.333333333333371,22.91666666666796,255.7777777777959,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
7,01,02,366817,0,13.333333333333371,29.59375000000314,386.80555555559874,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
8,01,02,366740,0,13.333333333333371,29.59375000000268,386.8055555555925,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False
9,01,02,367438,1,13.333333333333371,4.596254759978564,53.50561902193658,0,13.333333333333371,From 1 to 2,Not Set,False,interior,False,interior,False


In [13]:
# Getting Door information 
with open("..\SPARQL\doors.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Prepare the results for DataFrame
data = []
for row in results:
    door_url = str(row.door) if row and row.door else ""
    level = str(row.levelValue) if row and row.levelValue else ""
    mark = str(row.mark) if row and row.mark else ""
    fire_rating = str(row.fireRating) if row and row.fireRating else ""
    egress_type = str(row.egressType) if row and row.egressType else ""
    id = str(row.id) if row and row.id else ""
    base_elevation = str(row.baseElevationValue) if row and row.baseElevationValue else ""
    height = str(row.heightValue) if row and row.heightValue else ""
    width = str(row.widthValue) if row and row.widthValue else ""
    area = str(row.areaValue) if row and row.areaValue else ""
    
    data.append([level, mark, fire_rating, egress_type, id, base_elevation, height, width, area])

df = pd.DataFrame(data, columns=["Level", "Mark", "Fire Rating", "Egress Type", "ID", "Base Height", "Height", "Width", "Area"])

df["From Room"] = None  
df["To Room"] = None 
df["Door Swings Towards"] = None 

display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

,Level,Mark,Fire Rating,Egress Type,ID,Base Height,Height,Width,Area,From Room,To Room,Door Swings Towards
0,01,1,0 hours,,378315,0,7.23,6.23,31.93,None,None,None
1,01,10,0 hours,,381636,0,7,3,34.37,None,None,None
2,02,100,0 hours,,407249,13.33,7,3,34.37,None,None,None
3,02,101,0 hours,,407257,13.33,7.23,6.23,31.93,None,None,None
4,02,102,0 hours,,407258,13.33,7,3,34.37,None,None,None
5,02,103,0 hours,,407266,13.33,7.23,6.23,31.93,None,None,None
6,02,104,0 hours,,407267,13.33,7,3,34.37,None,None,None
7,02,105,0 hours,,407275,13.33,7.23,6.23,31.93,None,None,None
8,02,106,0 hours,,407276,13.33,7,3,34.37,None,None,None
9,02,107,0 hours,,407284,13.33,7.23,6.23,31.93,None,None,None


In [23]:
# Getting Windows information 
sparql = SPARQLWrapper("http://localhost:3030/SmartReviewHotel/sparql")  # Update with your Fuseki endpoint URL

sparql.setQuery(open("..\SPARQL\windows.txt", "r").read())
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

data = []
for row in results["results"]["bindings"]:  # Adjust if results format differs
    window_id = row["window"]["value"] if "window" in row else ""
    level = row["levelValue"]["value"] if "levelValue" in row else ""
    mark = row["mark"]["value"] if "mark" in row else ""
    type_ = row["type"]["value"] if "type" in row else ""
    fireRating = row["fireRating"]["value"] if "fireRating" in row else ""
    height = float(row["heightValue"]["value"]) if "heightValue" in row else 0.0
    width = float(row["widthValue"]["value"]) if "widthValue" in row else 0.0
    area = float(row["areaValue"]["value"]) if "areaValue" in row else 0.0
    baseHeight = (float(row["storeyHeightValue"]["value"]) + float(row["sillHeightValue"]["value"])) if "storeyHeightValue" in row and "sillHeightValue" in row else 0.0
    topHeight = baseHeight + height

    data.append([level, mark, type_, fireRating, height, width, area, baseHeight, topHeight])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Story", "Mark", "Name", "Fire Rating", "Height", "Width", "Area", "Base Height", "Top Height"])

df["Validly Hosted"] = True  # Adjust based on your conditions

# Display the DataFrame
display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

,Story,Mark,Name,Fire Rating,Height,Width,Area,Base Height,Top Height,Validly Hosted
0,02,49,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
1,02,50,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
2,02,47,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
3,02,48,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
4,02,45,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
5,02,46,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
6,02,44,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
7,02,51,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
8,02,39,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True
9,02,40,SMRV 36 x 72,1,6.000000,3.000000,31.465278,14.333333,20.333333,True


In [25]:
# Getting Stairs information 
with open("..\SPARQL\\stairs.sparql", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Pandas DataFrame
data = []
for row in results:
    data.append([str(row.label), str(row.baseValue), str(row.topValue), str(row.stairType), str(row.name)])

df = pd.DataFrame(data, columns=["ID", "Base Story", "Top Story", "Stair Type", "Name"])

df["Mark"] = "Not Set"  
df["Room Containg Stair"] = None 

display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

,ID,Base Story,Top Story,Stair Type,Name,Mark,Room Containg Stair
0,415870,01,02,Interior Exit Stairway,7 max riser 11 tread,Not Set,None
1,417752,02,03,Interior Exit Stairway,7 max riser 11 tread,Not Set,None
2,420117,02,03,Interior Exit Stairway,7 max riser 11 tread,Not Set,None
3,420699,03,04,Interior Exit Stairway,7 max riser 11 tread,Not Set,None
4,418246,03,04,Interior Exit Stairway,7 max riser 11 tread,Not Set,None
5,419050,01,02,Interior Exit Stairway,7 max riser 11 tread,Not Set,None


In [35]:
# Getting Areas information 
with open("..\SPARQL\\exitTravelDist.sparql", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Pandas DataFrame
data = []
for row in results:
    data.append([str(row.levelValue), str(row.name), str(row.number), str(row.exitValue), str(row.commonPathValue), str(row.smrvCalcOccupantLoadValue)])

df = pd.DataFrame(data, columns=["Storey", "Name", "Number", "Exit Travel Distance", "Common Path Distance", "Accumulated Occupant Load"])

display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

,Storey,Name,Number,Exit Travel Distance,Common Path Distance,Accumulated Occupant Load
0,01,King Suite,495,60,40,2
1,01,King Suite,495,60,40,2
2,01,King Suite,494,63,41,2
3,01,King Suite,494,63,41,2
4,01,1 Bedroom Suite - ADA,497,65,32,2
5,01,1 Bedroom Suite - ADA,497,65,32,2
6,01,1 Bedroom Suite - ADA,496,62,33,2
7,01,1 Bedroom Suite - ADA,496,62,33,2
8,01,King Suite,489,46,42,2
9,01,King Suite,489,46,42,2


In [28]:
# Getting Floor information 
with open("..\SPARQL\\floors.sparql", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

data = []
for row in results:
    floor_url = str(row.floor) if row and row.floor else ""
    level = str(row.levelValue) if row and row.levelValue else ""
    id = str(row.id) if row and row.id else ""
    height = str(row.heightValue) if row and row.heightValue else ""
    type = str(row.type) if row and row.type else ""
    area = str(row.areaValue) if row and row.areaValue else ""
    
    data.append([level, id, height, type, area])

# Create a DataFrame
df = pd.DataFrame(data, columns=[ "Story", "ID", "Height", "Type", "Area"])

df["Fire Rating"] = None  

# Display the DataFrame
display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

,Story,ID,Height,Type,Area,Fire Rating
0,01,364363,0,Floor: SMRV Generic - 12 1 hour,23252.810520637257,None
1,02,364513,13.333333333333371,Floor: SMRV Generic - 12 1 hour,15300.955986429224,None
2,03,364585,24.33333333333337,Floor: SMRV Generic - 12 1 hour,15072.367444762418,None
3,04,364631,35.33333333333337,Floor: SMRV Generic - 12 1 hour,15072.367444762418,None


In [32]:
# Getting Floor information 
with open("..\SPARQL\\roofs.txt", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

data = []
for row in results:
    roof_url = str(row.roof) if row and row.roof else ""
    id = str(row.id) if row and row.id else ""
    baseLevel = str(row.baseLevel) if row and row.baseLevel else ""
    type = str(row.type) if row and row.type else ""
    area = str(row.areaValue) if row and row.areaValue else ""
    
    data.append([id, baseLevel, type, area])

# Create a DataFrame
df = pd.DataFrame(data, columns=["ID", "Base Level", "Type", "Area"])

df["Fire Rating"] = None  
df["Flame Spread"] = None 

# Display the DataFrame
display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))

,ID,Base Level,Type,Area,Fire Rating,Flame Spread
0,366026,02-SECOND FLOOR,Basic Roof: SMRV Generic - 12 1 hour,2225.109485601434,None,None
1,365700,02-SECOND FLOOR,Basic Roof: SMRV Generic - 12 1 hour,6053.55061296662,None,None
2,444450,06-Machine Room Roof,Basic Roof: SMRV Generic - 12 1 hour,152.03691142465848,None,None
3,365632,05-Roof,Basic Roof: SMRV Generic - 12 1 hour,15654.29769280468,None,None
